In [ ]:
# ! pip install cvxpy
# ! pip install networkx

In [ ]:
# Import packages.
import cvxpy as cp
import numpy as np
from numpy.linalg import pinv, inv, norm
from scipy.linalg import eig
import networkx as nx

In [ ]:
n = 3
p = 100

W = np.random.rand(n, n)
W = (W + W.T) / 2
D = np.diag(np.sum(W, axis=0))
L_orig = D - W 
L_orig = L_orig / np.trace(L_orig) * n
[Lam, U] = eig(L_orig)
Lam, U = Lam.real, U.real

mu = np.zeros((n))
sigma = pinv(np.diag(Lam))
h = np.random.multivariate_normal(mu, sigma, p).T

sigma_eps = 1e-5
X = U @ h + np.random.multivariate_normal(np.zeros((n)), sigma_eps*np.eye(n), p).T

In [ ]:
def gl_sigrep(X, alpha, beta, num_iters):

  def optimize_L(Y):
    L = cp.Variable((n,n), symmetric=True)
    
    obj = cp.Minimize(alpha * cp.trace(Y.T @ L @ Y) + beta * (cp.norm(L, p='fro')**2))
    
    constraints = [cp.trace(L) == n]
    constraints += [L >> 0]
    constraints += [L @ np.ones((n)) == np.zeros((n))]
    for i in range(n):
      for j in range(i+1,n):
        constraints += [L[i,j] <= 0]

    prob = cp.Problem(obj, constraints)
    
    p_star = prob.solve()
    L_opt = L.value
    return L_opt

  def optimize_Y(L):

    # Y = cp.Variable((n, p))

    # obj = cp.Minimize((cp.norm(X - Y, p='fro')**2) + alpha * cp.trace(Y.T @ L @ Y))
    # prob = cp.Problem(obj)
    
    # p_star = prob.solve()
    # Y_opt = Y.value
    Y_opt = inv(np.eye(n) + alpha * L) @ X

    return Y_opt

  Y_iter = X
  obj_old_val = np.inf
  for idx_iter in np.arange(num_iters):
    L_iter = optimize_L(Y_iter)
    Y_iter = optimize_Y(L_iter)
    obj_val = norm(X - Y_iter, 'fro')**2 + alpha * np.trace(Y_iter.T @ L_iter @ Y_iter) + beta * (norm(L_iter, 'fro')**2);
    
    print(f"iteration-{idx_iter}: obj value = {obj_val}")

    if np.abs(obj_old_val - obj_val) < 1e-8:
      break
    else:
      obj_old_val = obj_val
  
  return L_iter, Y_iter

In [ ]:
alpha = 1e-2
beta = 10**(-0.2)
num_iters = 10
L_opt, Y_opt = gl_sigrep(X, alpha=alpha, beta=beta, num_iters=num_iters)

iteration-0: obj value = 5.113223138037308
iteration-1: obj value = 5.113186871633895
iteration-2: obj value = 5.11318678924065
iteration-3: obj value = 5.113186788387651


In [ ]:
print(L_orig)
print(L_opt)

[[ 0.74123708 -0.52012587 -0.22111121]
 [-0.52012587  1.27888879 -0.75876292]
 [-0.22111121 -0.75876292  0.97987413]]
[[ 0.89559205 -0.56581007 -0.32978548]
 [-0.56581007  1.17021883 -0.60441649]
 [-0.32978548 -0.60441649  0.93419853]]
